<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Get profile posts engagements
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Send_weekly_post_engagement_metrics_by_email.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #linkedin #posts #interactions #metrics #analytics #automation #naas #hubspot

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

## Input

### Get common variables, functions

In [5]:
# Except allow to run common.ipynb once using Naas Engine
try:
    %run "../common.ipynb"
except:
    %run "common.ipynb"

/home/ftp/Naas Content Engine


### Setup Variables

In [2]:
# Inputs
INPUT_DATABASE = LK_PROFILE_POSTS
OWNER = LK_FULLNAME
OWNER_ID = LK_PROFILE_ID
INPUT_LIKES = LK_PROFILE_POSTS_LIKES
INPUT_COMMENTS = LK_PROFILE_POSTS_COMMENTS
PERIOD = PERIOD_MTD

# Outputs
OUTPUT_DATABASE = LK_PROFILE_ENGAGEMENTS

## Model

### Get your posts

In [3]:
df_posts = get_data(INPUT_DATABASE)
print("✅ Posts fetched:", len(df_posts))
df_posts.head(1)

✅ Posts fetched: 296


,ACTIVITY_ID,PAGINATION_TOKEN,PUBLISHED_DATE,AUTHOR_NAME,AUTHOR_URL,SUBDESCRIPTION,TITLE,TEXT,CHARACTER_COUNT,TAGS,...,POLL_ID,POLL_QUESTION,POLL_RESULTS,POST_URL,VIEWS,COMMENTS,LIKES,SHARES,ENGAGEMENT_SCORE,DATE_EXTRACT
0,6946906609415544832,dXJuOmxpOmFjdGl2aXR5OjY5NDY5MDY2MDk0MTU1NDQ4Mz...,2022-06-26 19:26:42+02:00,Jérémy Ravenel,https://www.linkedin.com/in/ACoAAAJHE7sB5OxuKH...,20 hours ago,"If you are interested in data and business, yo...","If you are interested in data and business, yo...",1711,#66daysofdata #data #business #sales #people #...,...,NaN,NaN,NaN,https://www.linkedin.com/feed/update/urn:li:ac...,3153,1,26,0,0.0086,2022-06-27 18:07:27


### Get who likes your posts

In [4]:
df_likes = get_data(INPUT_LIKES)
print("✅ Likes fetched:", len(df_likes))
df_likes.head(1)

✅ Likes fetched: 5170


,PROFILE_ID,PROFILE_URL,PUBLIC_ID,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,PROFILE_PICTURE,BACKGROUND_PICTURE,PROFILE_TYPE,REACTION_TYPE,POST_URL,DATE_EXTRACT,TITLE,AUTHOR_NAME,PUBLISHED_DATE
0,ACoAAChHnMwBQ2zQadJd9cuma2eplAgjzGgew_A,https://www.linkedin.com/in/ACoAAChHnMwBQ2zQad...,amir-memon-55039916a,Amir,Memon,Amir Memon,Graduate Data Scientist👨‍💻 by profession ◾ Pet...,https://media-exp2.licdn.com/dms/image/C4E03AQ...,https://media-exp2.licdn.com/dms/image/C5616AQ...,PROFILE,PRAISE,https://www.linkedin.com/feed/update/urn:li:ac...,2022-06-28 17:36:36,🎨✍️Back at the design board.,Jérémy Ravenel,2022-06-20 20:50:56+02:00


### Get who comments your posts

In [5]:
df_comments = get_data(INPUT_COMMENTS)
print("✅ Comments fetched:", len(df_comments))
df_comments.head(1)

✅ Comments fetched: 395


,PROFILE_ID,PROFILE_URL,PUBLIC_ID,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,PROFILE_PICTURE,BACKGROUND_PICTURE,PROFILE_TYPE,...,CREATED_TIME,LANGUAGE,DISTANCE,COMMENTS,LIKES,POST_URL,DATE_EXTRACT,TITLE,AUTHOR_NAME,PUBLISHED_DATE
0,ACoAAAABPb8BgKkixeVN7cfz8qcUC28KUHQBfg4,https://www.linkedin.com/in/ACoAAAABPb8BgKkixe...,jeremywischusen,Jeremy,W.,Jeremy W.,Senior Full Stack Developer - The opinions I e...,https://media-exp2.licdn.com/dms/image/C5603AQ...,NaN,PROFILE,...,2022-06-24 05:05:45,English,DISTANCE_2,0,2,https://www.linkedin.com/feed/update/urn:li:ac...,2022-06-28 17:37:14,What's so hard about applied AI?,Jérémy Ravenel,2022-06-23 19:23:41+02:00


## Output

In [6]:
# Get interactions
def get_interactions(interaction, post_url):
    df = pd.DataFrame()
    try:
        if interaction == "LIKES":
            df = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(post_url)
        elif interaction == "COMMENTS":
            df = linkedin.connect(LI_AT, JSESSIONID).post.get_comments(post_url)
    except Exception as e:
        if e.response.status_code:
            print(e)
    return df

In [7]:
def update_interactions(df_posts,
                        df_interaction,
                        interaction,
                        csv_output,
                        no_posts=10,
                        min_updated_time=300):
    # Init
    df_out = df_interaction.copy()
    
    # Get all interactions if dataframe init empty or not complete
    if len(df_posts) == 0:
        return pd.DataFrame()

    if len(df_out) > 0:
        if "DATE_EXTRACT" in df_out.columns:
            last_update_date = df_out["DATE_EXTRACT"].max()
            time_last_update = datetime.now() - datetime.strptime(last_update_date, "%Y-%m-%d %H:%M:%S")
            minute_last_update = time_last_update.total_seconds() / 60
            if minute_last_update > min_updated_time:
                df_posts = df_posts[:no_posts]
            else:
                print(f"🛑 Nothing to update. Last update done {int(minute_last_update)} minutes ago.")
                return df_out.reset_index(drop=True)
    else:
        df_posts["SCENARIO"] = pd.to_datetime(df_posts["PUBLISHED_DATE"].str[:-6]).dt.strftime(PERIOD)
        df_posts = df_posts[df_posts["SCENARIO"] == datetime.now().strftime(PERIOD)].reset_index(drop=True)
        
    # Loop on posts
    for index, row in df_posts.iterrows():
        df_update = pd.DataFrame()
        post_title = row.TITLE
        post_author = row.AUTHOR_NAME
        post_url = row.POST_URL
        post_date = row.PUBLISHED_DATE
        count_interactions = row[interaction]
        print(f"🔄 {index+1} - Update started on: '{post_title}' ({post_url})")
        
        # Get interactions from post URL
        if len(df_interaction) > 0:
            tmp_df = df_interaction[df_interaction.POST_URL == post_url]
            no_interactions = len(tmp_df)
            if count_interactions != no_interactions:
                print(f"--> {count_interactions} post interaction count vs {no_interactions} interactions.")
                df_update = get_interactions(interaction, post_url)
            else:
                print("--->🛑 Nothing to update.")
        else:
            df_update = get_interactions(interaction, post_url)
        
        # Concat dataframe and save dataframe in CSV
        if len(df_update) > 0:
            print(f"---> {len(df_update)} interactions fetched.")
            df_update['TITLE'] = post_title
            df_update['AUTHOR_NAME'] = post_author
            df_update['PUBLISHED_DATE'] = post_date
            keys = ["POST_URL", "PROFILE_ID"]
            if interaction == "COMMENTS":
                keys = ["POST_URL", "PROFILE_ID", "CREATED_TIME"]
            df_out = pd.concat([df_update, df_out]).drop_duplicates(keys, keep="first")
            output_path = save_data(df_out, csv_output)
            
    # Add dependency in production
    print(f"✅ {len(df_out)} '{interaction}' fetched.")
    # Return all interactions
    return df_out.reset_index(drop=True)

### Update likes
It will update your last 10 posts like's from LinkedIn API.<br>
PS: On the first execution all posts like's will be retrieved.

In [8]:
df_update_likes = update_interactions(df_posts,
                                      df_likes,
                                      "LIKES",
                                      INPUT_LIKES)
df_update_likes.head(1)

🛑 Nothing to update. Last update done 222 minutes ago.


,PROFILE_ID,PROFILE_URL,PUBLIC_ID,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,PROFILE_PICTURE,BACKGROUND_PICTURE,PROFILE_TYPE,REACTION_TYPE,POST_URL,DATE_EXTRACT,TITLE,AUTHOR_NAME,PUBLISHED_DATE
0,ACoAAChHnMwBQ2zQadJd9cuma2eplAgjzGgew_A,https://www.linkedin.com/in/ACoAAChHnMwBQ2zQad...,amir-memon-55039916a,Amir,Memon,Amir Memon,Graduate Data Scientist👨‍💻 by profession ◾ Pet...,https://media-exp2.licdn.com/dms/image/C4E03AQ...,https://media-exp2.licdn.com/dms/image/C5616AQ...,PROFILE,PRAISE,https://www.linkedin.com/feed/update/urn:li:ac...,2022-06-28 17:36:36,🎨✍️Back at the design board.,Jérémy Ravenel,2022-06-20 20:50:56+02:00


### Update comments
It will update your last 10 posts comment's from LinkedIn API.<br>
PS: On the first execution all posts comment's will be retrieved.

In [9]:
df_update_comments = update_interactions(df_posts,
                                         df_comments,
                                         "COMMENTS",
                                         INPUT_COMMENTS)
df_update_comments.head(1)

🛑 Nothing to update. Last update done 221 minutes ago.


,PROFILE_ID,PROFILE_URL,PUBLIC_ID,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,PROFILE_PICTURE,BACKGROUND_PICTURE,PROFILE_TYPE,...,CREATED_TIME,LANGUAGE,DISTANCE,COMMENTS,LIKES,POST_URL,DATE_EXTRACT,TITLE,AUTHOR_NAME,PUBLISHED_DATE
0,ACoAAAABPb8BgKkixeVN7cfz8qcUC28KUHQBfg4,https://www.linkedin.com/in/ACoAAAABPb8BgKkixe...,jeremywischusen,Jeremy,W.,Jeremy W.,Senior Full Stack Developer - The opinions I e...,https://media-exp2.licdn.com/dms/image/C5603AQ...,NaN,PROFILE,...,2022-06-24 05:05:45,English,DISTANCE_2,0,2,https://www.linkedin.com/feed/update/urn:li:ac...,2022-06-28 17:37:14,What's so hard about applied AI?,Jérémy Ravenel,2022-06-23 19:23:41+02:00


### Create interactions database
- Concat LIKES and COMMENTS in single database
- Create note for HubSpot

In [10]:
def create_interactions_db(df_likes, df_comments):
    # Init outputs
    df = pd.DataFrame()
    
    # Dataframe likes
    df_likes["REACTION"] = "LIKES"

    # Dataframe comments
    df_comments["REACTION"] = "COMMENTS"
    
    # Concat
    df = pd.concat([df_likes, df_comments]).fillna("Not defined").sort_values(by="PUBLISHED_DATE", ascending=False)
    
    # Cleaning
    to_keep = [
        "PROFILE_ID",
        "PROFILE_URL",
        "PUBLIC_ID",
        "FIRSTNAME",
        "LASTNAME",
        "FULLNAME",
        "OCCUPATION",
        "REACTION",
        "TEXT",
        "TITLE",
        "PUBLISHED_DATE",
        "AUTHOR_NAME",
        "POST_URL",
    ]
    df = df[to_keep]
    
    print(f"✅ {len(df)} interactions fetched.")
    return df.reset_index(drop=True)

df_interactions = create_interactions_db(df_update_likes, df_update_comments)
df_interactions.head(3)

✅ 5565 interactions fetched.


,PROFILE_ID,PROFILE_URL,PUBLIC_ID,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,REACTION,TEXT,TITLE,PUBLISHED_DATE,AUTHOR_NAME,POST_URL
0,ACoAAB5zKVIB37Fjtw_HTJEcClYvE8CgL5Z9zUE,https://www.linkedin.com/in/ACoAAB5zKVIB37Fjtw...,ariprabowo,Ari Sulistiyo,Prabowo,Ari Sulistiyo Prabowo,Data & Business | Entrepreneur | Investor,COMMENTS,Nice sharing Jérémy Ravenel,"If you are interested in data and business, yo...",2022-06-26 19:26:42+02:00,Jérémy Ravenel,https://www.linkedin.com/feed/update/urn:li:ac...
1,ACoAAAW5nOsBFnNwysowDAZ9t87XbBuKTo47tsE,https://www.linkedin.com/in/ACoAAAW5nOsBFnNwys...,syree,Dr. Syreeta,Charles-Cole,Dr. Syreeta Charles-Cole,Interests | Ethical AI & Machine Learning,LIKES,Not defined,"If you are interested in data and business, yo...",2022-06-26 19:26:42+02:00,Jérémy Ravenel,https://www.linkedin.com/feed/update/urn:li:ac...
2,ACoAAADS0WQBhQQVMD2eFJNZgjOOjDTH6ptbScU,https://www.linkedin.com/in/ACoAAADS0WQBhQQVMD...,vineetvashishta,Vin,Vashishta,Vin Vashishta,Chief Data Officer | AI Strategist | DataScien...,LIKES,Not defined,"If you are interested in data and business, yo...",2022-06-26 19:26:42+02:00,Jérémy Ravenel,https://www.linkedin.com/feed/update/urn:li:ac...


### Save data

In [11]:
save_data(df_interactions, OUTPUT_DATABASE)

👌 Well done! Your Dependency has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
✅ Dataframe successfully saved in CSV: LinkedIn/Inputs/LINKEDIN_PROFILE_ENGAGEMENTS_ACoAAAJHE7sB5OxuKHuzguZ9L6lfDHqw--cdnJg.csv
